In [1]:
import cv2
import numpy as np

In [177]:
def center_point(x,y,w,h):
    return int(x) + int(w/2), int(y) + int(h/2)


def is_in_offset(center_point, line_point_1, line_point_2, offset):                              
    tg = (line_point_1[1] - line_point_2[1]) / (line_point_1[0] - line_point_2[0])
    ly = offset / pow(1 + tg, 1/2)
    lx = pow(offset ** 2 - ly ** 2, 1/2)
    p1 = (line_point_1[0] + lx, line_point_1[1] - ly)
    p2 = (line_point_2[0] + lx, line_point_2[1] - ly)
    p3 = (line_point_2[0] - lx, line_point_2[1] + ly)
    p4 = (line_point_1[0] - lx, line_point_1[1] + ly)
    xp = int(center_point[0])
    yp = int(center_point[1])
    is_in = False
    if (p2[0] - p1[0]) * (yp - p1[1]) - (xp - p1[0]) * (p2[1] - p1[1]) >= 0 \
        and (p3[0] - p2[0]) * (yp - p2[1]) - (xp - p2[0]) * (p3[1] - p2[1]) >= 0 \
        and (p4[0] - p3[0]) * (yp - p3[1]) - (xp - p3[0]) * (p4[1] - p3[1]) >= 0 \
        and (p1[0] - p4[0]) * (yp - p4[1]) - (xp - p4[0]) * (p1[1] - p4[1]) >= 0:
        is_in = True
    return is_in


def rect_node(frame, line_point_1, line_point_2, offset):
    tg = (line_point_1[1] - line_point_2[1]) / (line_point_1[0] - line_point_2[0])
    ly = offset / pow(1 + tg, 1/2)
    lx = pow(offset ** 2 - ly ** 2, 1/2)
    p1 = (int(line_point_1[0] + lx), int(line_point_1[1] - ly))
    p2 = (int(line_point_2[0] + lx), int(line_point_2[1] - ly))
    p3 = (int(line_point_2[0] - lx), int(line_point_2[1] + ly))
    p4 = (int(line_point_1[0] - lx), int(line_point_1[1] + ly))
    return [p1,p2,p3,p4]

In [178]:
cap = cv2.VideoCapture('crossroad.mp4')

counter = 0
min_width = 60
min_height = 60
detected = []
count_line_1 = (275,275)
count_line_2 = (525, 575)
offset = 5
back_substractor = cv2.bgsegm.createBackgroundSubtractorMOG()

while(cap.isOpened()):
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (3,3), 5)
    
    img_sub = back_substractor.apply(blur)
    kernel_dilate = np.ones((5,5), 'uint8')
    dilated = cv2.dilate(img_sub, kernel_dilate)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(5,5))
    dst_0 = cv2.morphologyEx(dilated, cv2.MORPH_CLOSE, kernel)
    dst = cv2.morphologyEx(dst_0, cv2.MORPH_CLOSE, kernel)
    contours, h = cv2.findContours(dst, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    #cv2.imshow('Detector', dst)
    
    cv2.line(frame, (275,275), (525, 575), (248,24,48), 3)
    
    for i in rect_node(frame, count_line_1, count_line_2, offset):
        cv2.circle(frame, i, 3, (255,0,0), -1)
    
    for (i, shape) in enumerate(contours):
        (x,y,w,h) = cv2.boundingRect(shape)
        is_big_enough = w > min_width and h > min_height
        if not is_big_enough:
            continue
    
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2)
        center = center_point(x,y,w,h)
        cv2.circle(frame, center,3,(34,54,180),-1)
        detected.append(center)                                        
    for (x,y) in detected:
        if is_in_offset((x,y), count_line_1, count_line_2, offset):
            counter += 1
            print(counter, (x,y))
            cv2.line(frame, count_line_1, count_line_2, (0,25,158), 5)
            detected.remove((x,y))
            
    cv2.putText(frame, "vehicles so far: " + str(counter), (25,25), cv2.FONT_HERSHEY_COMPLEX, fontScale=1, color=(200,10,20))

    cv2.imshow('Crossroad_video', frame)
    if cv2.waitKey(1) == 1:
        break
    
cap.release()
cv2.destroyAllWindows()

1 (413, 434)
2 (342, 359)
3 (416, 450)
4 (357, 373)
5 (363, 376)
6 (366, 379)
7 (358, 381)
8 (370, 384)
9 (335, 351)
10 (377, 390)
11 (367, 393)
12 (326, 332)
13 (321, 334)
14 (328, 340)
15 (326, 332)
16 (323, 334)


KeyboardInterrupt: 